#PROJETO BASE:
Este projeto é baseado no código desenvolvido por Anderson Carvalho. Projeto Original: https://github.com/anderoak/projeto_consultor_ai

#Consulta documentos de IA
Este notebook demonstra um projeto que combina a API Gemini com embeddings para criar um chatbot capaz de responder perguntas sobre o conteúdo de documentos e buscar informações relevantes em uma base de documentos PDF.

###Instruções para Utilização

-  Adicione a sua chave de API do Google AI no Secrets, na barra lateral esquerda do Colab, definindo com o nome: GEMINI_API_KEY.
-  Crie uma pasta chamada PDF no ambiente Colab. Você pode fazer clicando no ícone arquivos na lateral esquerda do Colab, depois clicando com o botão direito no espaço em branco e selecionando "Nova pasta".
- Depois faça upload dos seus arquivos PDF e arraste-os para dentro da pasta PDF que criou. Leve em conta que a API do Gemini pode limitar de quantidade de dados carregados dos PDF se você está usando a versão gratuita da API, então use apenas um ou dois arquivos pequenos (deixei 2 arquivos que usei para testar no [repositório do GitHub](https://github.com/anderoak/projeto_consultor_ai), na pasta "PDFs para teste").
- Garanta que os arquivos na pasta sejam realmente PDFs. Atente que esse projeto assume que os PDFs contêm texto extraível, portanto não irá funcionar com PDFs que são apenas imagens digitalizadas, pois exigiria técnicas de OCR (Optical Character Recognition) para extrair o texto.
-  Execute o código célula por célula.
-  Interaja com o chatbot fazendo perguntas sobre o conteúdo dos PDFs.
-  Digite "sair" para encerrar o chat.

Observação: Este é um exemplo básico genérico, mas que talvez possa se aprimorado adicionando mais funcionalidades, como especialização por área e uma interface web.

##1. Instalação de Bibliotecas

In [1]:
# Instala as bibliotecas necessárias
!pip install -q -U google-generativeai pypdf2

# Importa as bibliotecas
import google.generativeai as genai
from google.colab import userdata
import numpy as np
import PyPDF2

# Lê a API Key do Secrets do Colab
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


## Funções para processar PDFs

In [3]:
def extrair_texto_pdf(caminho_arquivo):
  """
  Extrai o texto de um arquivo PDF.
  """
  with open(caminho_arquivo, 'rb') as arquivo_pdf:
    leitor_pdf = PyPDF2.PdfReader(arquivo_pdf)
    texto = ""
    for num_pagina in range(len(leitor_pdf.pages)):
      pagina = leitor_pdf.pages[num_pagina]
      texto += pagina.extract_text()
    return texto

def carregar_documentos_pdf(pasta_colab='./'):
  """
  Carrega os arquivos PDF de uma pasta no Colab e extrai o texto.
  """
  import os
  arquivos_pdf = [os.path.join(pasta_colab, arquivo)
                   for arquivo in os.listdir(pasta_colab)
                   if arquivo.endswith('.pdf')]

  # Extrai o texto de cada arquivo PDF
  documentos = [extrair_texto_pdf(arquivo) for arquivo in arquivos_pdf]
  return documentos

##2. Configuração do Modelo Gemini

In [4]:
# Configura os parâmetros do modelo Gemini
generation_config = {
    "candidate_count": 1,  # Número de respostas a serem geradas
    "temperature": 0.5,   # Controla a criatividade das respostas
}

# Configura os parâmetros de segurança
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

# Inicializa o modelo Gemini
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              generation_config=generation_config,
                              safety_settings=safety_settings)

# Define o modelo de embedding
embedding_model = "models/embedding-001"

##3. Carregamento da Base de Dados e Embeddings

In [5]:
# Carrega os documentos da pasta do Colab
documentos = carregar_documentos_pdf()

# Função para dividir o texto em chunks menores
def dividir_texto_em_chunks(texto, tamanho_chunk=5000): # tamanho do chunk em caracteres
  """Divide um texto em chunks menores."""
  return [texto[i:i + tamanho_chunk] for i in range(0, len(texto), tamanho_chunk)]

# Gera embeddings para os documentos usando Gemini, dividindo em chunks se necessário
embeddings_documentos = []
for documento in documentos:
  chunks = dividir_texto_em_chunks(documento)  # Divide o documento em chunks
  embeddings_documento = []
  for chunk in chunks:
    embedding = genai.embed_content(model=embedding_model,
                                    content=chunk,  # Usa o chunk em vez do documento inteiro
                                    task_type="RETRIEVAL_DOCUMENT")["embedding"]
    embeddings_documento.extend(embedding)  # Adiciona o embedding do chunk à lista
  embeddings_documentos.append(embeddings_documento)  # Adiciona todos os embeddings do documento à lista principal

## 4. Função de Busca por Similaridade

In [6]:
import numpy as np

def buscar_documento(pergunta, embeddings_documentos, documentos):
  """
  Busca o documento mais similar à pergunta usando embeddings.
  """
  # Gera o embedding da pergunta usando Gemini
  embedding_pergunta = genai.embed_content(model=embedding_model,
                                           content=pergunta,
                                           task_type="RETRIEVAL_QUERY")["embedding"]

  # Calcula a similaridade cosseno entre a pergunta e os documentos
  embedding_pergunta = np.array(embedding_pergunta)
  similaridades = []
  for documento_embeddings in embeddings_documentos:
    documento_embeddings = np.array(documento_embeddings)

    # Calculate average embedding if necessary
    if len(documento_embeddings.shape) > 1 and documento_embeddings.shape[0] != 768:
        documento_embeddings = np.mean(documento_embeddings, axis=0)

    # Ensure document_embeddings has shape (1, 768) or (768,)
    if documento_embeddings.shape != (768,) and documento_embeddings.shape != (1, 768):
        print("Warning: Document embedding shape is not (768,) or (1, 768):", documento_embeddings.shape)
        # Handle the issue appropriately, e.g., by skipping the document or using a default similarity
        continue  # Skip this document and continue with the next

    # Reshape to (1, 768) if necessary to enable broadcasting
    if documento_embeddings.shape == (768,):
        documento_embeddings = documento_embeddings.reshape(1, -1)

    # Calculate cosine similarity using broadcasting
    similaridade = np.mean(np.sum(documento_embeddings * embedding_pergunta.reshape(1, -1), axis=1) / (np.linalg.norm(documento_embeddings, axis=1) * np.linalg.norm(embedding_pergunta)))
    similaridades.append(similaridade)

  # Encontra o índice do documento mais similar
  # Return the first document if similaridades is empty to avoid the error
  indice_documento_similar = np.argmax(similaridades) if similaridades else 0

  # Retorna o documento mais similar
  return documentos[indice_documento_similar]

## 5. Interação com o Chatbot

In [ ]:
# Inicia o chat
chat = model.start_chat(history=[])

# Loop de interação
while True:
  # Obtém a pergunta do usuário
  pergunta = input("Pergunta: ")

  # Busca o documento mais similar
  documento_similar = buscar_documento(pergunta, embeddings_documentos, documentos)

  # Envia a pergunta e o documento para o Gemini
  resposta = chat.send_message(f"Pergunta: {pergunta}\nContexto: {documento_similar}")

  # Exibe a resposta do Gemini
  print(f"Resposta: {resposta.text}")

  # Condição de parada
  if pergunta.lower() == 'sair':
    break

Resposta: As glosas que se referem à negociação de serviço são:

* **Glosa: 022 – Negociação Serv. não permite solicitar serviços:**  Esta glosa indica que o prestador de serviço informado na solicitação não pode solicitar serviços, devido à configuração da negociação de serviço.

* **Glosa: 023 – Negociação Serv. não permite solicitar serviços para Tipo Local:**  Esta glosa indica que o prestador de serviço informado não pode solicitar serviços para o tipo de local especificado, conforme a negociação de serviço.

* **Glosa: 083 – Negociação Tab. Serv. para Contrato/Modelo não encontrada:** Esta glosa indica que não foi encontrada uma negociação de tabela de serviço vinculada ao contrato ou modelo de contrato do beneficiário.

* **Glosa: 084 – Negociação Tab. Serv. para Prestador/ClPrestador não encontrada:** Esta glosa indica que não foi encontrada uma negociação de tabela de serviço vinculada ao prestador de serviço ou classe de prestador.

* **Glosa: 113 – Item exige Indicação Clíni